# **Proyecto Equipo 8. Detección temprana de riesgo de suicidio**

In [ ]:
# Importar e instalar dependencias
import pandas as pd
import numpy as np
import re
from collections import Counter

# CARGA DE DATOS

In [ ]:
!pip install gdown
# Descarga el archivo del dataset de drive usando gdown
url = 'https://drive.google.com/file/d/1z3fmc6QKE71_OQ3Poy-fTq1DpJcoK-Z_/view?usp=drive_link'
file_id = url.split('/')[-2]
!gdown --id $file_id

# Carga el dataset usando pandas
sd = pd.read_csv('/content/Suicide_Detection_clean_2.csv')

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1z3fmc6QKE71_OQ3Poy-fTq1DpJcoK-Z_
From (redirected): https://drive.google.com/uc?id=1z3fmc6QKE71_OQ3Poy-fTq1DpJcoK-Z_&confirm=t&uuid=7f0bac94-f4ff-4a20-8ce8-f6401f0897d4
To: /content/Suicide_Detection_clean_2.csv
100% 227M/227M [00:01<00:00, 143MB/s]


# MODELADO, ENTRENAMIENTO Y EVALUACIÓN

## Modelo DistilBERT

In [ ]:
!pip install transformers torch evaluate datasets optuna
import optuna
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertConfig, EarlyStoppingCallback
import torch
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Asumimos que 'sd' y 'pre_processed' ya están definidos
# Sampleo de data para agilizar el testeo del código
sd = sd.sample(n=20000, random_state=42)
pre_processed = sd.copy()

# Separación de dataset en training y testing
train_data, test_data = train_test_split(pre_processed, test_size=0.2, random_state=42)

# Crear un encoder para las etiquetas
label_encoder = LabelEncoder()

# Ajustar el encoder a las etiquetas y transformar
train_data['class'] = label_encoder.fit_transform(train_data['class'])
test_data['class'] = label_encoder.transform(test_data['class'])

# Cargar el tokenizador DistilBERT
model_name = 'distilbert-base-uncased'
tokenizer_bert = DistilBertTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)

def encode_data(examples):
    return tokenizer_bert(examples['text'], truncation=True, padding='max_length', max_length=128)

# Convertir los datos a datasets de Hugging Face
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Aplicar la codificación a los datasets
train_dataset = train_dataset.map(encode_data, batched=True)
test_dataset = test_dataset.map(encode_data, batched=True)

# Renombrar la columna 'class' a 'labels'
train_dataset = train_dataset.rename_column('class', 'labels')
test_dataset = test_dataset.rename_column('class', 'labels')

# Configurar el formato de los datasets
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Función para calcular métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
def objective(trial):
    # Definir los hiperparámetros a optimizar
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 8)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64])
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.0, 0.2)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)
    hidden_dropout_prob = trial.suggest_float("hidden_dropout_prob", 0.1, 0.5)
    attention_probs_dropout_prob = trial.suggest_float("attention_probs_dropout_prob", 0.1, 0.5)

    # Configuración personalizada del modelo DistilBERT
    custom_config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
    custom_config.hidden_dropout_prob = hidden_dropout_prob
    custom_config.attention_probs_dropout_prob = attention_probs_dropout_prob
    custom_config.num_labels = 2

    # Cargar el modelo DistilBERT con la configuración personalizada
    model = DistilBertForSequenceClassification.from_pretrained(model_name, config=custom_config)

    # Configurar argumentos de entrenamiento
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=64,
        learning_rate=learning_rate,
        warmup_ratio=warmup_ratio,
        weight_decay=weight_decay,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    # Entrenador
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Entrenar el modelo
    trainer.train()

    # Evaluar el modelo
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

# Crear un estudio de Optuna

# Create or load an Optuna study using a persistent storage (SQLite database in this case)
study_name = "my_study"  # Choose a name for your study
storage_name = f"sqlite:///{study_name}.db"  # Define the storage location
study = optuna.create_study(
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,  # Load if the study already exists
    direction="maximize"
)

study.optimize(objective, n_trials=5)  # Puedes ajustar el número de trials según tus necesidades

print("Mejor trial:")
trial = study.best_trial

print("Valor:", trial.value)
print("Mejores hiperparámetros:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Entrenar el modelo final con los mejores hiperparámetros
best_params = study.best_params

# Configuración final del modelo
final_config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
final_config.hidden_dropout_prob = best_params["hidden_dropout_prob"]
final_config.attention_probs_dropout_prob = best_params["attention_probs_dropout_prob"]
final_config.num_labels = 2

final_model = DistilBertForSequenceClassification.from_pretrained(model_name, config=final_config)

final_training_args = TrainingArguments(
    output_dir='./final_results',
    num_train_epochs=best_params["num_train_epochs"],
    per_device_train_batch_size=best_params["per_device_train_batch_size"],
    per_device_eval_batch_size=64,
    learning_rate=best_params["learning_rate"],
    warmup_ratio=best_params["warmup_ratio"],
    weight_decay=best_params["weight_decay"],
    logging_dir='./final_logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Entrenar el modelo final
final_trainer.train()

# Evaluar el modelo final
final_eval_results = final_trainer.evaluate()

print("Resultados finales de evaluación:")
print(final_eval_results)

# Realizar predicciones en el conjunto de prueba
predictions = final_trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=-1)

accuracy = accuracy_score(test_dataset['labels'], predicted_labels)
precision = precision_score(test_dataset['labels'], predicted_labels, average='weighted')
recall = recall_score(test_dataset['labels'], predicted_labels, average='weighted')
f1 = f1_score(test_dataset['labels'], predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")



[I 2024-10-17 14:25:56,878] A new study created in RDB with name: my_study


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.139500,0.132626,0.954250,0.954245,0.954304,0.954250
2,0.134500,0.116149,0.961500,0.961499,0.961509,0.961500
3,0.085500,0.130674,0.961000,0.960994,0.961110,0.961000
4,0.062300,0.145272,0.961250,0.961245,0.961328,0.961250
5,0.039400,0.158712,0.961250,0.961249,0.961254,0.961250


[I 2024-10-17 14:41:30,858] Trial 0 finished with value: 0.9614985559250832 and parameters: {'learning_rate': 1.0746566580811717e-05, 'num_train_epochs': 6, 'per_device_train_batch_size': 32, 'warmup_ratio': 0.1523165249347074, 'weight_decay': 0.05388053983243142, 'hidden_dropout_prob': 0.16050991134845016, 'attention_probs_dropout_prob': 0.3727671631916408}. Best is trial 0 with value: 0.9614985559250832.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.163300,0.145708,0.948750,0.948709,0.949653,0.948750
2,0.083000,0.119005,0.956500,0.956469,0.957285,0.956500
3,0.064500,0.161144,0.951750,0.951691,0.953268,0.951750
4,0.023900,0.195353,0.955000,0.954970,0.955744,0.955000
5,0.007200,0.181515,0.964750,0.964751,0.964841,0.964750
6,0.002000,0.202846,0.963250,0.963251,0.963267,0.963250
7,0.000400,0.204974,0.962500,0.962499,0.962501,0.962500


[I 2024-10-17 15:22:50,867] Trial 2 finished with value: 0.9647513681858606 and parameters: {'learning_rate': 3.242765985747258e-05, 'num_train_epochs': 7, 'per_device_train_batch_size': 64, 'warmup_ratio': 0.15388053592928025, 'weight_decay': 0.0512759710884516, 'hidden_dropout_prob': 0.4812207762931957, 'attention_probs_dropout_prob': 0.3955136133368543}. Best is trial 2 with value: 0.9647513681858606.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.156500,0.151070,0.954250,0.954211,0.955251,0.954250
2,0.068700,0.127139,0.962750,0.962739,0.962995,0.962750
3,0.009300,0.178044,0.959750,0.959730,0.960250,0.959750
4,0.028300,0.202301,0.963250,0.963250,0.963250,0.963250
5,0.053800,0.207633,0.961500,0.961498,0.961514,0.961500


[I 2024-10-17 15:39:53,904] Trial 3 finished with value: 0.9632501125637666 and parameters: {'learning_rate': 1.9620272496607295e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.04329566469747129, 'weight_decay': 0.06828769888705809, 'hidden_dropout_prob': 0.3162826509014548, 'attention_probs_dropout_prob': 0.3310290203935703}. Best is trial 2 with value: 0.9647513681858606.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.142100,0.134172,0.956250,0.956233,0.956606,0.956250
2,0.072200,0.134366,0.962000,0.961993,0.962141,0.962000
3,0.055500,0.185309,0.960500,0.960476,0.961148,0.960500
4,0.036800,0.194369,0.962500,0.962498,0.962519,0.962500
